In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import matplotlib
import pandas
import numpy
print (matplotlib.__version__)
print (pandas.__version__)
print (numpy.__version__)

1.5.3
0.18.1
1.11.1


In [3]:
#Reading users file:
#users_data = pd.read_csv('/home/nadine/Desktop/Memoir/data/user.csv')

In [4]:
#Reading jobs file:
#jobs_data = pd.read_csv('/home/nadine/Desktop/Memoir/data/jobs.csv')


In [5]:
#Reading ratings file:
df = pd.read_csv('/home/serge/Desktop/Kanye/Hubert/Memoir/data/data/data.csv')

In [6]:
# Now lets take a peak into the content of each file to understand them better.

In [7]:
# To get the size of rating's data
#print (rating.shape)
# To get the firts five users!
#rating.head()
#len(rating.UserId)

In [8]:
# Now we have to divide the ratings data set into test and train data for making models:
# To split the rating data into traing and test data, we can use "scikit-learn library"

In [9]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.15)
tn_data  = pd.DataFrame(train_data, columns=['UserId','JobId','Rating'])
ts_data  = pd.DataFrame(test_data, columns=['UserId','JobId','Rating'])


In [10]:
# COLLABORATIVE FILTERING ALGORITHMS

In [11]:
## 1. Memory-Based Collaborative Filtering:

In [12]:
n_users = df.UserId.unique().shape[0]
n_jobs = df.JobId.unique().shape[0]
print (' Total Number of users = ' + str(n_users) + ' | Total Number of jobs = ' + str(n_jobs) )
n_users

 Total Number of users = 2500 | Total Number of jobs = 150


2500

In [13]:
# 1. User_Item Filtering Algorithm: 
## A user-item filtering will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked
##  Main steps
### Build user-item matrix: X12 represents rating from user1 to job2
### Calculate the simmility: user simalality based on ratings simality and job similarity based  on ratings simalarity using cosine similarity
### Create the simility matrix

In [14]:
#tn_data['JobId']

In [15]:
### Build user-item matrix for Traing dataset
train_data_matrix = np.zeros((n_users, n_jobs))
for line in tn_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    
### Build user-item matrix for Testing dataset
test_data_matrix = np.zeros((n_users, n_jobs))
for line in ts_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]   

In [16]:
#train_data_matrix,test_data_matrix,
train_data_matrix[:1,:2]

array([[ 4.,  0.]])

In [17]:
### Compute the cosine similarity
## A distance metric commonly used in Recommender system is 
## cosine simmilarity and uses rating as a n-dimensional matrix
## and the similarity is calculated based on the angle between these vectors.

In [18]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [19]:
print(item_similarity[:4, :4])
print(user_similarity[:4, :4])

[[  0.00000000e+00   6.78086377e-01   6.46514728e-01   6.41090955e-01]
 [  6.78086377e-01  -6.66133815e-16   6.49085406e-01   6.30914869e-01]
 [  6.46514728e-01   6.49085406e-01   2.22044605e-16   6.21846667e-01]
 [  6.41090955e-01   6.30914869e-01   6.21846667e-01   2.22044605e-16]]
[[  2.22044605e-16   6.37880584e-01   7.13820668e-01   7.06056305e-01]
 [  6.37880584e-01   2.22044605e-16   5.36852714e-01   6.45417119e-01]
 [  7.13820668e-01   5.36852714e-01   0.00000000e+00   6.42755562e-01]
 [  7.06056305e-01   6.45417119e-01   6.42755562e-01   1.11022302e-16]]


In [20]:
data_sims = pd.DataFrame(user_similarity)
data_sims.ix[:1, :1]

,0,1
0,2.220446e-16,6.378806e-01
1,6.378806e-01,2.220446e-16


In [21]:
# Let us check the computational time
%timeit pairwise_distances(train_data_matrix, metric='cosine')

1 loop, best of 3: 1.09 s per loop


In [22]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'job':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [23]:
# with the calculated simalarity matrix now we can use it to predict tyhe non rated jobs offers!

In [24]:
item_prediction = predict(train_data_matrix, item_similarity, type='job')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [25]:
%timeit predict(train_data_matrix, item_similarity, type='job')
%timeit predict(train_data_matrix, user_similarity, type='user')

10 loops, best of 3: 97.1 ms per loop
1 loop, best of 3: 1.21 s per loop


In [26]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [27]:
print 'User-based CF RMSE for Training Set: ' + str(rmse(user_prediction, train_data_matrix))
print 'User-based CF RMSE for Testing Set: ' + str(rmse(user_prediction, test_data_matrix))

print 'Item-based CF RMSE for Training Set: ' + str(rmse(item_prediction, train_data_matrix))
print 'Item-based CF RMSE for Testing Set: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE for Training Set: 2.48793993578
User-based CF RMSE for Testing Set: 2.48840146542
Item-based CF RMSE for Training Set: 2.48960129307
Item-based CF RMSE for Testing Set: 2.48956347757


In [28]:
# Singular Value Decomposition

In [29]:
sparsity=round(1.0-len(df)/float(n_users*n_jobs),3)
print 'The sparsity level of WutikoData is ' +  str(sparsity*100) + '%'

The sparsity level of WutikoData is 46.7%


In [30]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print 'User-based CF MSE for Training Set: ' + str(rmse(X_pred, train_data_matrix))
print 'User-based CF MSE for Testing Set: ' + str(rmse(X_pred, test_data_matrix))

User-based CF MSE for Training Set: 2.14233675377
User-based CF MSE for Testing Set: 2.60676794482


In [31]:
## Content-Boosted Collaborative Filtering:
# This approach uses a content-based predictor to enhance existing user data,
# and then provides personalized suggestions through collaborative filtering
# Content-Boosted Collaborative Filtering, performs better than a pure content-based predictor, pure
# collaborative filter, and a naive hybrid approach.

In [32]:
# Model Evaluation
# The metrics for evaluating the accuracy of a prediction algorithm can be divided into two main categories: 
# statistical accuracy metrics and decision-support metrics.
# 1. Statistical accuracy metrics evaluate the accu-
# racy of a predictor by comparing predicted values with user-
#provided values. To measure statistical accuracy we use the
# mean absolute error (MAE) metric — defined as the average
# absolute difference between predicted ratings and actual rat-
# ings. In our experiments we computed the MAE on the test
# set for each user, and then averaged over the set of test users

In [33]:

# 2.Decision-support accuracy measures how well predic-
# tions help users select high-quality items. We use Re-
# ceiver Operating Characteristic (ROC) sensitivity to mea-
# sure decision-support accuracy. A predictor can be treated as a filter, where predicting a high rating for an item is
# equivalent to accepting the item, and predicting a low rating
# is equivalent to rejecting the item. The ROC sensitivity is
# given by the area under the ROC curve — a curve that plots
# sensitivity versus 1-specificity for a predictor. Sensitivity is
# defined as the probability that a good item is accepted by the
# filter; and specificity is defined as the probability that a bad
# item is rejected by the filter. We consider an item good if the
# user gave it a rating of 4 or above, otherwise we consider the
# item bad.

In [34]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print 'User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix))

User-based CF MSE: 2.60676794482


In [35]:
# Tutorials

In [36]:
def compute_rmse(y_pred, y_true):
    """Computr Root Mean Squared Error."""
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [37]:
def evaluate(estimate_f):
    """RMSE-based predictive performance evaulate with pandas"""
    ids_to_estimate = zip(ts_data.UserId, ts_data.JobId)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = ts_data.Rating.values
    return compute_rmse(estimated, real)

In [38]:
def my_estimate_func(UserId, JobId):
    return 3.0

In [39]:
print('RMSE for estimate: %s' % evaluate(my_estimate_func)) 

RMSE for estimate: 1.29874854814


In [40]:
# Simple Collaborative

In [41]:
def collab_mean(UserId, JobId):
    """Simple content-filtering based on mean ratings."""
    user_condition = tn_data.UserId == UserId
    job_condition = tn_data.JobId == JobId
    ratings_by_others = tn_data.loc[user_condition & job_condition]
    if ratings_by_others.empty:
        return 3.0
    else:  
        return ratings_by_others.Rating.mean()

print 'RMSE for estiamte1: %s' % evaluate(collab_mean)

RMSE for estiamte1: 1.45787962921


In [42]:
# Content-based Filtering using mean ratings

In [43]:
def content_mean(UserId, JobId):
    """Simple content-filtering based on mean ratings."""
    user_condition = tn_data.UserId == UserId
    return tn_data.loc[user_condition, 'Rating'].mean()
print 'RMSE for estiamte2: %s' % evaluate(content_mean)

RMSE for estiamte2: 1.25161157425
